In [116]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [118]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [120]:
def calculate_angle_2d(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [122]:
def calculate_distance_2d(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

In [124]:
def normalize_keypoints(keypoints):
    if keypoints is None:
        return None
    # Use raw coordinates (0-1 range) directly without excessive normalization for visualization
    return keypoints

In [125]:
def get_keypoints_2d(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if not results.pose_landmarks:
        print("No landmarks detected in image.")
        return None
    landmarks = results.pose_landmarks.landmark
    keypoints = {
        'nose': (landmarks[mp_pose.PoseLandmark.NOSE].x, landmarks[mp_pose.PoseLandmark.NOSE].y),
        'left_shoulder': (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y),
        'right_shoulder': (landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y),
        'left_elbow': (landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y),
        'right_elbow': (landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y),
        'left_wrist': (landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y),
        'right_wrist': (landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y),
        'left_hip': (landmarks[mp_pose.PoseLandmark.LEFT_HIP].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP].y),
        'right_hip': (landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y),
        'left_knee': (landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y),
        'right_knee': (landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE].y),
        'left_ankle': (landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y),
        'right_ankle': (landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y),
    }
    return keypoints

In [152]:
pose_references = {
    "vriksasana": r"F:\BRiX\Yoga-Dataset-By-Sabree\vriksasana.jpg",
    "stand" : r"D:\1600px_COLOURBOX18738680.jpg",
    "classic" : r"D:\c.jpg"
}

In [130]:
def build_pose_database_from_images(pose_references):
    pose_database = {}
    for pose_name, img_path in pose_references.items():
        ref_image = cv2.imread(img_path)
        if ref_image is None:
            print(f"Warning: Could not load image for {pose_name} at {img_path}")
            continue
        keypoints = get_keypoints_2d(ref_image)
        if keypoints is None:
            print(f"Warning: Could not detect keypoints for {pose_name}")
            continue
        pose_database[pose_name] = keypoints
        print(f"Added {pose_name} to the database.")
    return pose_database


In [132]:
def draw_custom_landmarks(image, keypoints, incorrect_points):
    h, w = image.shape[:2]
    for point_name, (x, y) in keypoints.items():
        color = (0, 255, 0) if point_name not in incorrect_points else (0, 0, 255)  # Green for correct, Red for incorrect
        x_clipped = np.clip(x, 0, 1)
        y_clipped = np.clip(y, 0, 1)
        cv2.circle(image, (int(x_clipped * w), int(y_clipped * h)), 5, color, -1)

In [133]:
import cv2
import numpy as np

def draw_reference_dummy(image, ref_keypoints):
    h, w = image.shape[:2]
    dummy_size = 200
    dummy = np.zeros((dummy_size, dummy_size, 3), dtype=np.uint8)

    # Find min/max of keypoints to scale them within dummy
    x_vals = [kp[0] for kp in ref_keypoints.values()]
    y_vals = [kp[1] for kp in ref_keypoints.values()]
    x_min, x_max = min(x_vals), max(x_vals)
    y_min, y_max = min(y_vals), max(y_vals)
    x_range = x_max - x_min if x_max > x_min else 1
    y_range = y_max - y_min if y_max > y_min else 1

    scaled_keypoints = {}

    # Normalize and scale keypoints to fit within dummy
    for point_name, (x, y) in ref_keypoints.items():
        x_norm = (x - x_min) / x_range
        y_norm = (y - y_min) / y_range
        x_scaled = int(x_norm * (dummy_size - 10)) + 5  # Leave 5px padding
        y_scaled = int(y_norm * (dummy_size - 10)) + 5
        scaled_keypoints[point_name] = (x_scaled, y_scaled)

    # Draw connections (tiny lines between keypoints)
    connections = [
        ("left_shoulder", "right_shoulder"),
        ("left_shoulder", "left_elbow"),
        ("left_elbow", "left_wrist"),
        ("right_shoulder", "right_elbow"),
        ("right_elbow", "right_wrist"),
        ("left_hip", "right_hip"),
        ("left_shoulder", "left_hip"),
        ("right_shoulder", "right_hip"),
        ("left_hip", "left_knee"),
        ("left_knee", "left_ankle"),
        ("right_hip", "right_knee"),
        ("right_knee", "right_ankle"),
    ]

    for p1, p2 in connections:
        if p1 in scaled_keypoints and p2 in scaled_keypoints:
            cv2.line(dummy, scaled_keypoints[p1], scaled_keypoints[p2], (200, 200, 200), 1)

    # Draw keypoints
    for _, (x, y) in scaled_keypoints.items():
        cv2.circle(dummy, (x, y), 3, (255, 255, 255), -1)

    # Place dummy on the image
    image[10:10+dummy_size, w-dummy_size-10:w-10] = dummy

In [134]:
def generate_pose_specific_checks(pose_database):
    pose_specific_checks = {}
    for pose_name, ref_keypoints in pose_database.items():
        checks = []
        
        # Leg Straightness
        left_leg_angle = calculate_angle_2d(ref_keypoints['left_hip'], ref_keypoints['left_knee'], ref_keypoints['left_ankle'])
        if left_leg_angle > 150:
            checks.append(('left_knee', 
                          lambda ref, user: calculate_angle_2d(ref['left_hip'], ref['left_knee'], ref['left_ankle']),
                          lambda ref, user: calculate_angle_2d(user['left_hip'], user['left_knee'], user['left_ankle']),
                          lambda diff, ref_val: "Straighten your left leg fully." if diff > ref_val * 0.3 else None, 1.0))
        elif left_leg_angle < 90:
            checks.append(('left_knee', 
                          lambda ref, user: calculate_angle_2d(ref['left_hip'], ref['left_knee'], ref['left_ankle']),
                          lambda ref, user: calculate_angle_2d(user['left_hip'], user['left_knee'], user['left_ankle']),
                          lambda diff, ref_val: "Bend your left knee more." if diff < -ref_val * 0.3 else None, 1.0))
        
        right_leg_angle = calculate_angle_2d(ref_keypoints['right_hip'], ref_keypoints['right_knee'], ref_keypoints['right_ankle'])
        if right_leg_angle > 150:
            checks.append(('right_knee', 
                          lambda ref, user: calculate_angle_2d(ref['right_hip'], ref['right_knee'], ref['right_ankle']),
                          lambda ref, user: calculate_angle_2d(user['right_hip'], user['right_knee'], user['right_ankle']),
                          lambda diff, ref_val: "Straighten your right leg fully." if diff > ref_val * 0.3 else None, 1.0))
        elif right_leg_angle < 90:
            checks.append(('right_knee', 
                          lambda ref, user: calculate_angle_2d(ref['right_hip'], ref['right_knee'], ref['right_ankle']),
                          lambda ref, user: calculate_angle_2d(user['right_hip'], user['right_knee'], user['right_ankle']),
                          lambda diff, ref_val: "Bend your right knee more." if diff < -ref_val * 0.3 else None, 1.0))
        
        # Hip Height
        if ref_keypoints['left_hip'][1] < 0.3:
            checks.append(('left_hip', 
                          lambda ref, user: ref['left_hip'][1],
                          lambda ref, user: user['left_hip'][1],
                          lambda diff, ref_val: "Raise your hips higher." if diff > ref_val * 0.3 else None, 0.9))
        elif ref_keypoints['left_hip'][1] > 0.7:
            checks.append(('left_hip', 
                          lambda ref, user: ref['left_hip'][1],
                          lambda ref, user: user['left_hip'][1],
                          lambda diff, ref_val: "Lower your hips closer to the ground." if diff < -ref_val * 0.3 else None, 0.9))
        
        # Arm Straightness
        left_arm_angle = calculate_angle_2d(ref_keypoints['left_shoulder'], ref_keypoints['left_elbow'], ref_keypoints['left_wrist'])
        if left_arm_angle > 150:
            checks.append(('left_elbow', 
                          lambda ref, user: calculate_angle_2d(ref['left_shoulder'], ref['left_elbow'], ref['left_wrist']),
                          lambda ref, user: calculate_angle_2d(user['left_shoulder'], user['left_elbow'], user['left_wrist']),
                          lambda diff, ref_val: "Straighten your left arm fully." if diff > ref_val * 0.3 else None, 0.8))
        
        right_arm_angle = calculate_angle_2d(ref_keypoints['right_shoulder'], ref_keypoints['right_elbow'], ref_keypoints['right_wrist'])
        if right_arm_angle > 150:
            checks.append(('right_elbow', 
                          lambda ref, user: calculate_angle_2d(ref['right_shoulder'], ref['right_elbow'], ref['right_wrist']),
                          lambda ref, user: calculate_angle_2d(user['right_shoulder'], user['right_elbow'], user['right_wrist']),
                          lambda diff, ref_val: "Straighten your right arm fully." if diff > ref_val * 0.3 else None, 0.8))
        
        # Foot Position
        if ref_keypoints['left_ankle'][1] < ref_keypoints['right_ankle'][1] - 0.1:
            checks.append(('left_ankle', 
                          lambda ref, user: ref['left_ankle'][1],
                          lambda ref, user: user['left_ankle'][1],
                          lambda diff, ref_val: "Lift your left foot higher." if diff > ref_val * 0.3 else None, 0.7))
        elif ref_keypoints['right_ankle'][1] < ref_keypoints['left_ankle'][1] - 0.1:
            checks.append(('right_ankle', 
                          lambda ref, user: ref['right_ankle'][1],
                          lambda ref, user: user['right_ankle'][1],
                          lambda diff, ref_val: "Lift your right foot higher." if diff > ref_val * 0.3 else None, 0.7))
        
        pose_specific_checks[pose_name] = checks
    return pose_specific_checks

In [146]:
def get_all_feedback(reference_keypoints, user_keypoints, pose_name, pose_specific_checks):
    if reference_keypoints is None or user_keypoints is None:
        return ["No pose detected"], set()
    
    feedback_list = []
    incorrect_points = set()
    error_threshold = 0.4  # 30% error rate

    general_checks = [
        ('right_knee', lambda ref, user: calculate_angle_2d(ref['right_hip'], ref['right_knee'], ref['right_ankle']),
         lambda ref, user: calculate_angle_2d(user['right_hip'], user['right_knee'], user['right_ankle']),
         lambda diff, ref_val: "Straighten your right leg fully." if diff > ref_val * error_threshold and ref_val > 150 else
                              "Bend your right knee more." if diff < -ref_val * error_threshold and ref_val < 90 else None, 1.0),
        ('left_knee', lambda ref, user: calculate_angle_2d(ref['left_hip'], ref['left_knee'], ref['left_ankle']),
         lambda ref, user: calculate_angle_2d(user['left_hip'], user['left_knee'], user['left_ankle']),
         lambda diff, ref_val: "Straighten your left leg fully." if diff > ref_val * error_threshold and ref_val > 150 else
                              "Bend your left knee more." if diff < -ref_val * error_threshold and ref_val < 90 else None, 1.0),
        ('right_ankle', lambda ref, user: calculate_distance_2d(ref['right_hip'], ref['right_ankle']),
         lambda ref, user: calculate_distance_2d(user['right_hip'], user['right_ankle']),
         lambda diff, ref_val: "Extend your right leg further." if diff > ref_val * error_threshold else
                              "Bring your right leg closer." if diff < -ref_val * error_threshold else None, 0.9),
        ('left_ankle', lambda ref, user: calculate_distance_2d(ref['left_hip'], ref['left_ankle']),
         lambda ref, user: calculate_distance_2d(user['left_hip'], user['left_ankle']),
         lambda diff, ref_val: "Extend your left leg further." if diff > ref_val * error_threshold else
                              "Bring your left leg closer." if diff < -ref_val * error_threshold else None, 0.9),
        ('left_ankle', lambda ref, user: abs(ref['left_ankle'][0] - ref['right_ankle'][0]),
         lambda ref, user: abs(user['left_ankle'][0] - user['right_ankle'][0]),
         lambda diff, ref_val: "Widen your stance." if diff > ref_val * error_threshold else
                              "Narrow your stance." if diff < -ref_val * error_threshold else None, 0.8),
        ('left_ankle', lambda ref, user: ref['left_ankle'][1],
         lambda ref, user: user['left_ankle'][1],
         lambda diff, ref_val: "Lift your left foot higher." if diff > ref_val * error_threshold else
                              "Lower your left foot." if diff < -ref_val * error_threshold else None, 0.7),
        ('right_ankle', lambda ref, user: ref['right_ankle'][1],
         lambda ref, user: user['right_ankle'][1],
         lambda diff, ref_val: "Lift your right foot higher." if diff > ref_val * error_threshold else
                              "Lower your right foot." if diff < -ref_val * error_threshold else None, 0.7),
        ('left_hip', lambda ref, user: abs(ref['left_hip'][1] - ref['right_hip'][1]),
         lambda ref, user: abs(user['left_hip'][1] - user['right_hip'][1]),
         lambda diff, ref_val: "Level your hips." if diff > ref_val * error_threshold else None, 0.9),
        ('left_hip', lambda ref, user: ref['left_hip'][1],
         lambda ref, user: user['left_hip'][1],
         lambda diff, ref_val: "Raise your hips higher." if diff > ref_val * error_threshold else
                              "Lower your hips." if diff < -ref_val * error_threshold else None, 0.8),
        ('left_hip', lambda ref, user: ref['left_hip'][0],
         lambda ref, user: user['left_hip'][0],
         lambda diff, ref_val: "Shift your hips left." if diff > ref_val * error_threshold else
                              "Shift your hips right." if diff < -ref_val * error_threshold else None, 0.7),
        ('left_shoulder', lambda ref, user: calculate_angle_2d(ref['left_shoulder'], ref['left_hip'], ref['left_ankle']),
         lambda ref, user: calculate_angle_2d(user['left_shoulder'], user['left_hip'], user['left_ankle']),
         lambda diff, ref_val: "Straighten your torso." if diff > ref_val * error_threshold else
                              "Bend your torso more." if diff < -ref_val * error_threshold else None, 0.9),
        ('right_shoulder', lambda ref, user: calculate_angle_2d(ref['right_shoulder'], ref['right_hip'], ref['right_ankle']),
         lambda ref, user: calculate_angle_2d(user['right_shoulder'], user['right_hip'], user['right_ankle']),
         lambda diff, ref_val: "Straighten your torso." if diff > ref_val * error_threshold else
                              "Bend your torso more." if diff < -ref_val * error_threshold else None, 0.9),
        ('right_elbow', lambda ref, user: calculate_angle_2d(ref['right_shoulder'], ref['right_elbow'], ref['right_wrist']),
         lambda ref, user: calculate_angle_2d(user['right_shoulder'], user['right_elbow'], user['right_wrist']),
         lambda diff, ref_val: "Straighten your right arm." if diff > ref_val * error_threshold and ref_val > 150 else
                              "Bend your right elbow more." if diff < -ref_val * error_threshold and ref_val < 90 else None, 0.8),
        ('left_elbow', lambda ref, user: calculate_angle_2d(ref['left_shoulder'], ref['left_elbow'], ref['left_wrist']),
         lambda ref, user: calculate_angle_2d(user['left_shoulder'], user['left_elbow'], user['left_wrist']),
         lambda diff, ref_val: "Straighten your left arm." if diff > ref_val * error_threshold and ref_val > 150 else
                              "Bend your left elbow more." if diff < -ref_val * error_threshold and ref_val < 90 else None, 0.8),
        ('right_wrist', lambda ref, user: calculate_distance_2d(ref['right_shoulder'], ref['right_wrist']),
         lambda ref, user: calculate_distance_2d(user['right_shoulder'], user['right_wrist']),
         lambda diff, ref_val: "Extend your right arm further." if diff > ref_val * error_threshold else
                              "Bring your right arm closer." if diff < -ref_val * error_threshold else None, 0.7),
        ('left_wrist', lambda ref, user: calculate_distance_2d(ref['left_shoulder'], ref['left_wrist']),
         lambda ref, user: calculate_distance_2d(user['left_shoulder'], user['left_wrist']),
         lambda diff, ref_val: "Extend your left arm further." if diff > ref_val * error_threshold else
                              "Bring your left arm closer." if diff < -ref_val * error_threshold else None, 0.7),
        ('left_shoulder', lambda ref, user: abs(ref['left_shoulder'][1] - ref['right_shoulder'][1]),
         lambda ref, user: abs(user['left_shoulder'][1] - user['right_shoulder'][1]),
         lambda diff, ref_val: "Level your shoulders." if diff > ref_val * error_threshold else None, 0.8),
        ('left_shoulder', lambda ref, user: ref['left_shoulder'][1],
         lambda ref, user: user['left_shoulder'][1],
         lambda diff, ref_val: "Raise your shoulders higher." if diff > ref_val * error_threshold else
                              "Lower your shoulders." if diff < -ref_val * error_threshold else None, 0.7),
        ('left_shoulder', lambda ref, user: ref['left_shoulder'][0],
         lambda ref, user: user['left_shoulder'][0],
         lambda diff, ref_val: "Shift your shoulders left." if diff > ref_val * error_threshold else
                              "Shift your shoulders right." if diff < -ref_val * error_threshold else None, 0.6),
        ('nose', lambda ref, user: ref['nose'][1] - (ref['left_shoulder'][1] + ref['right_shoulder'][1]) / 2,
         lambda ref, user: user['nose'][1] - (user['left_shoulder'][1] + user['right_shoulder'][1]) / 2,
         lambda diff, ref_val: "Lift your head higher." if diff > ref_val * error_threshold else
                              "Lower your head." if diff < -ref_val * error_threshold else None, 0.6),
        ('nose', lambda ref, user: ref['nose'][0] - (ref['left_shoulder'][0] + ref['right_shoulder'][0]) / 2,
         lambda ref, user: user['nose'][0] - (user['left_shoulder'][0] + user['right_shoulder'][0]) / 2,
         lambda diff, ref_val: "Tilt your head left." if diff > ref_val * error_threshold else
                              "Tilt your head right." if diff < -ref_val * error_threshold else None, 0.5),
    ]

    checks = general_checks + pose_specific_checks.get(pose_name, [])
    for point_name, ref_fn, user_fn, feedback_fn, weight in checks:
        ref_val = ref_fn(reference_keypoints, user_keypoints)
        user_val = user_fn(reference_keypoints, user_keypoints)
        diff = ref_val - user_val
        feedback = feedback_fn(diff, ref_val)
        if feedback:
            feedback_list.append((feedback, weight))
            incorrect_points.add(point_name)

    feedback_list.sort(key=lambda x: x[1], reverse=True)
    return [f[0] for f in feedback_list], incorrect_points if feedback_list else ([f"Excellent {pose_name}!"], set())


In [154]:
import cv2
import time
import pyautogui  # For getting screen resolution

# Get screen width and height
screen_width, screen_height = pyautogui.size()

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam.")

# Build pose database from provided references
pose_database = build_pose_database_from_images(pose_references)
if not pose_database:
    print("No valid poses loaded from the database. Exiting.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Get user input for the pose to perform
print("Available poses:", list(pose_database.keys()))
selected_pose = input("Enter the pose you want to perform: ").strip()
if selected_pose not in pose_database:
    print(f"Pose '{selected_pose}' not found in the database. Exiting.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

reference_keypoints = pose_database[selected_pose]
pose_specific_checks = generate_pose_specific_checks(pose_database)

feedback_list = []
incorrect_points = set()
current_feedback_idx = 0

# Create a full-screen window
cv2.namedWindow(f"{selected_pose} Correction", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty(f"{selected_pose} Correction", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from webcam.")
        break

    # Resize the frame to fit the screen
    frame = cv2.resize(frame, (screen_width, screen_height))

    user_keypoints = get_keypoints_2d(frame)
    if user_keypoints is None:
        cv2.putText(frame, "No pose detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow(f"{selected_pose} Correction", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    draw_reference_dummy(frame, reference_keypoints)
    draw_custom_landmarks(frame, user_keypoints, incorrect_points)
    feedback_list, incorrect_points = get_all_feedback(reference_keypoints, user_keypoints, selected_pose, pose_specific_checks)
    
    if feedback_list:
        current_feedback = feedback_list[min(current_feedback_idx, len(feedback_list)-1)]
        cv2.putText(frame, f"{selected_pose}: {current_feedback}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        temp_feedback, _ = get_all_feedback(reference_keypoints, user_keypoints, selected_pose, pose_specific_checks)
        if current_feedback not in temp_feedback:
            current_feedback_idx += 1
            time.sleep(0.5)
    else:
        cv2.putText(frame, f"{selected_pose} perfected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        time.sleep(2)
        feedback_list = []
        current_feedback_idx = 0

    cv2.imshow(f"{selected_pose} Correction", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


No landmarks detected in image.
Added stand to the database.
Added classic to the database.
Available poses: ['stand', 'classic']


Enter the pose you want to perform:  classic
